In [159]:
import re
import json

def extract_json_using_patterns(text):
    """使用一组正则表达式模式来提取 JSON"""
    # 保留原始文本的换行符
    text = text.strip()
    print("text:", text)
    
    patterns = [
        r'\{[\s\S]*\}',  # 新的正则表达式模式，匹配第一个 '{' 和最后一个 '}' 之间的内容
        # 保留其他原有模式
        r'(\{[\s\S]*?\})\s*\}$',
        r'\{\s*"result"\s*:\s*\[[\s\S]*?\]\s*\}',
        r'"""json\s*(\{[\s\S]*?\})\s*"""',
        r'```json\s*(\{[\s\S]*?\})\s*```',
    ]

    # 依次尝试使用每个正则表达式模式进行匹配
    for pattern in patterns:
        match = re.search(pattern, text, re.DOTALL)
        if match:
            # 检查正则表达式是否使用了捕获组
            if match.lastindex:
                json_str = match.group(1)  # 获取捕获组中的内容
            else:
                json_str = match.group(0)  # 获取整个匹配的内容
            print(f"匹配到的 JSON: {json_str}")
            try:
                # 尝试将提取到的字符串解析为 JSON
                result_data = json.loads(json_str)
                return result_data
            except json.JSONDecodeError as e:
                # 如果 JSON 解析失败，继续尝试下一个模式
                print(f"JSON 解析失败: {e}")
                continue

    # 如果没有找到匹配的 JSON 数据
    print("未找到符合模式的 JSON 数据")
    return {}

# 测试代码
text = '''
一些前置文本
{
    "result": [
        {"type": "发起", "content": "具你，为什么要我买？这是第一套。"},
        {"type": "其它", "content": ""}
    ],
    "compliance": "低"
}
一些后置文本
'''

result = extract_json_using_patterns(text)
print("提取结果:", result)


text: 一些前置文本
{
    "result": [
        {"type": "发起", "content": "具你，为什么要我买？这是第一套。"},
        {"type": "其它", "content": ""}
    ],
    "compliance": "低"
}
一些后置文本
匹配到的 JSON: {
    "result": [
        {"type": "发起", "content": "具你，为什么要我买？这是第一套。"},
        {"type": "其它", "content": ""}
    ],
    "compliance": "低"
}
提取结果: {'result': [{'type': '发起', 'content': '具你，为什么要我买？这是第一套。'}, {'type': '其它', 'content': ''}], 'compliance': '低'}


In [157]:
extract_json_using_patterns(text)

text: ```json
{
"result":
[{"type":"发起","content": "具你，为什么要我买？这是第一套。"},
{"type":"其它","content": ""}],
"compliance": "低"
}
匹配到的 JSON: {
"result":
[{"type":"发起","content": "具你，为什么要我买？这是第一套。"}
JSON 解析失败: Expecting ',' delimiter: line 3 column 45 (char 56)
未找到符合模式的 JSON 数据


{}

In [158]:
{
"result":
[{"type":"发起","content": "具你，为什么要我买？这是第一套。"},
{"type":"其它","content": ""}],
"compliance": "低"
}

{'result': [{'type': '发起', 'content': '具你，为什么要我买？这是第一套。'},
  {'type': '其它', 'content': ''}],
 'compliance': '低'}

In [176]:
import pandas as pd 
def parse_test_result(test_result):
    """解析模型返回的结果，提取 JSON"""
    if test_result:
        try:
            result_data = json.loads(test_result)
            
            return result_data
        except json.JSONDecodeError:
            return extract_json_using_patterns(test_result)
    else:
        return {}
def remove_punctuation(text):
    # 中英文标点符号的正则表达式
    punctuation_pattern = r'[，。！？、；：“”‘’（）《》【】…—]+' + r'[.,\/#!$%\^&\*;:{}=\-_`~()]'
    # 使用正则表达式替换标点符号为空字符串
    no_punctuation_text = re.sub(punctuation_pattern, '', text)
    return no_punctuation_text

output_result = [[{"start_time": 27300, "end_time": 32940, "text": "具你，为什么要我买？这是第一套。", "label": 0}, {"start_time": 35310, "end_time": 39510, "text": "喂，你，吃你吃你狗，你，", "label": 1}, {"model_input": "\n                “老师话语”：具你，为什么要我买？这是第一套。 \n                “学生话语”：喂，你，吃你吃你狗，你，\n                "}, {"soikit_test_result": "{\n  \"result\": [\n    {\"type\": \"发起\", \"content\": \"具你，为什么要我买？这是第一套。\"}\n  ],\n  \"compliance\": \"低\"\n}"}], [{"start_time": 40560, "end_time": 42710, "text": "好，把语文书翻到第50页，", "label": 0}, {"start_time": 40560, "end_time": 42710, "text": "", "label": 1}, {"model_input": "“老师话语”：好，把语文书翻到第50页， "}, {"soikit_test_result": "{\n  \"result\": [\n    {\"type\": \"其它\", \"content\": \"好，把语文书翻到第50页，\"}\n  ],\n  \"compliance\": \"无\"\n}"}], [{"start_time": 45590, "end_time": 47190, "text": "然后铅笔收起来把，", "label": 0}, {"start_time": 47910, "end_time": 49590, "text": "课堂练习放到左上角，", "label": 1}, {"model_input": "\n                “老师话语”：然后铅笔收起来把， \n                “学生话语”：课堂练习放到左上角，\n                "}, {"soikit_test_result": "{\n  \"result\": [\n    {\"type\": \"其它\", \"content\": \"然后铅笔收起来吧\"}\n  ],\n  \"compliance\": \"无\"\n}"}], [{"start_time": 50070, "end_time": 52760, "text": "先把语文书翻到翻到第50页，翻到这里，", "label": 0}, {"start_time": 52780, "end_time": 52790, "text": "没有，50。我现在这个阳猫世，", "label": 1}, {"model_input": "\n                “老师话语”：先把语文书翻到翻到第50页，翻到这里， \n                “学生话语”：没有，50。我现在这个阳猫世，\n                "}, {"soikit_test_result": "{\n  \"result\": [\n    {\"type\": \"其它\", \"content\": \"先把语文书翻到翻到第50页，翻到这里，\"}\n  ],\n  \"compliance\": \"无\"\n}"}], [{"start_time": 53000, "end_time": 69880, "text": "我看谁今天坐姿有问题啊啊，", "label": 0}, {"start_time": 53000, "end_time": 69880, "text": "", "label": 1}, {"model_input": "“老师话语”：我看谁今天坐姿有问题啊啊， "}, {"soikit_test_result": "{\n\"result\": [\n  {\"type\":\"发起\",\"content\":\"我看谁今天坐姿有问题啊啊，\"}\n],\n\"compliance\": \"无\"\n}"}]]
model_name = "soikit_test"
# def process_output_result(output_result, model_name):
"""处理模型的输出结果，生成最终的 DataFrame"""
result_list = []

for sublist in output_result:
    print("sub_list",sublist)
    test_result = None
    if len(sublist) >= 4:
        third_item = sublist[3]
        test_result = next(iter(third_item.values()))
    else:
        test_result = None
    
    contents = parse_test_result(test_result)
    print("contents",contents)
    new_sublist = []
    if not contents:
        for item in sublist:
            print("item",item)
            if 'start_time' in item:
                new_item = item.copy()
                new_item[f'{model_name}_predict'] = '{}'
                new_sublist.append(new_item)
        result_list.append(new_sublist)
        continue
    for item in sublist:
        if 'start_time' in item:
            new_item = item.copy()

            match_found = False
            for content in contents["result"]:
                if content['content'] != "":
                    if remove_punctuation(content['content']) in remove_punctuation(item['text']):
                        new_item[f'{model_name}_predict'] = test_result
                        match_found = True
                        break
            if not match_found:
                new_item[f'{model_name}_predict'] = ''

            new_sublist.append(new_item)
    result_list.append(new_sublist)
    # return df_result

sub_list [{'start_time': 27300, 'end_time': 32940, 'text': '具你，为什么要我买？这是第一套。', 'label': 0}, {'start_time': 35310, 'end_time': 39510, 'text': '喂，你，吃你吃你狗，你，', 'label': 1}, {'model_input': '\n                “老师话语”：具你，为什么要我买？这是第一套。 \n                “学生话语”：喂，你，吃你吃你狗，你，\n                '}, {'soikit_test_result': '{\n  "result": [\n    {"type": "发起", "content": "具你，为什么要我买？这是第一套。"}\n  ],\n  "compliance": "低"\n}'}]
contents {'result': [{'type': '发起', 'content': '具你，为什么要我买？这是第一套。'}], 'compliance': '低'}
sub_list [{'start_time': 40560, 'end_time': 42710, 'text': '好，把语文书翻到第50页，', 'label': 0}, {'start_time': 40560, 'end_time': 42710, 'text': '', 'label': 1}, {'model_input': '“老师话语”：好，把语文书翻到第50页， '}, {'soikit_test_result': '{\n  "result": [\n    {"type": "其它", "content": "好，把语文书翻到第50页，"}\n  ],\n  "compliance": "无"\n}'}]
contents {'result': [{'type': '其它', 'content': '好，把语文书翻到第50页，'}], 'compliance': '无'}
sub_list [{'start_time': 45590, 'end_time': 47190, 'text': '然后铅笔收起来把，', 'label': 0}, {'start_ti

In [174]:
sub_list  = [{'start_time': 27300, 'end_time': 32940, 'text': '具你，为什么要我买？这是第一套。', 'label': 0}, {'start_time': 35310, 'end_time': 39510, 'text': '喂，你，吃你吃你狗，你，', 'label': 1}, {'model_input': '\n                “老师话语”：具你，为什么要我买？这是第一套。 \n                “学生话语”：喂，你，吃你吃你狗，你，\n                '}, {'soikit_test_result': '{\n  "result": [\n    {"type": "发起", "content": "具你，为什么要我买？这是第一套。"}\n  ],\n  "compliance": "低"\n}'}]
sub_list[0]

{'start_time': 27300,
 'end_time': 32940,
 'text': '具你，为什么要我买？这是第一套。',
 'label': 0}

In [ ]:
if "start_time" in sub_list[0]:
    print(1)

In [ ]:
    


    for item in sublist:
        if 'start_time' in item:
            new_item = item.copy()

            match_found = False
            for content in contents["result"]:
                if content['content'] != "":
                    if content['content'] in item['text']:
                        new_item[f'{model_name}_predict'] = test_result
                        match_found = True
                        break
            if not match_found:
                new_item[f'{model_name}_predict'] = ''

            new_sublist.append(new_item)
    result_list.append(new_sublist)

flat_list = [item for sublist in result_list for item in sublist]
df_result = pd.DataFrame(flat_list)